In [ ]:
# 구글 드라이브 연동
from google.colab import drive

drive.mount('/content/drive')

# 드라이브 파일 목록 확인
!ls drive/'My Drive'/'Colab Notebooks'/

%cd drive/MyDrive/Colab Notebooks/VisionAI

!ls

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

#파이썬 take_photo 함수 
def take_photo(filename='photo.jpg', quality=0.8):
  #javascript 작성 시작
  js = Javascript('''
    async function takePhoto(quality) {
      
      //div(공간) 생성
      const div = document.createElement('div');

      //button 생성
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      //video 생성
      const video = document.createElement('video');
      //비디오 모양 네모네모
      video.style.display = 'block';
      //카메라(웹캠) 불러오기
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      //div 밑에 child 공간 추가
      document.body.appendChild(div);
      //공간에 video 넣기
      div.appendChild(video);
      //video와 웹캠 연결
      video.srcObject = stream;
      //await -> 비동기식 처리 (thread와 관련)(async와 세트)
      await video.play();

      // 비디오 요소에 맞게 출력 크기를 조정
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // capture 버튼을 클릭할 때까지 wait
      await new Promise((resolve) => capture.onclick = resolve);

      //canvas 생성
      const canvas = document.createElement('canvas');
      //크기 맞추기
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      //이미지 그리기
      canvas.getContext('2d').drawImage(video, 0, 0);
      //비디오 끄기
      stream.getVideoTracks()[0].stop();
      //div 삭제
      div.remove();
      // 파일 주소 반환
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # javascript 코드 실행한 결과(사진) 반환 값
  data = eval_js('takePhoto({})'.format(quality))
  
  # 웹 브라우저에서 데이터를 저장할때 base64로 저장
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
import os
import torch
import cv2
from utils import utils_image as util
from models.network_rrdbnet import RRDBNet as net

In [ ]:
model_path = os.path.join('model_zoo', 'BSRGAN.pth')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = net(in_nc=3, out_nc=3, nf=64, nb=23, gc=32, sf=4)

model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
for k, v in model.named_parameters():
    v.requires_grad = False
model = model.to(device)

[3, 3, 64, 23, 32, 4]


In [ ]:
def change_image(img):
    with torch.no_grad():
        img = cv2.imread(img)

        # RGB 형태의 3채널 이미지로 변환
        img_L = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)

        # 텐서 형태로 변환
        img_L = util.uint2tensor4(img_L)

        # Line 10 device 변수 참조
        img_L = img_L.to(device)

        # 모델을 통해 inference
        img_E = model(img_L)

        # 텐서를 이미지로 변환
        img_E = util.tensor2uint(img_E)

        # 이미지 저장
        return util.imshow(x=img_E, title="BSRGAN", cbar=False, figsize=(13, 13))

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # 방금 저장된 화면 사진 출력
  display(Image(filename))
  display(change_image(filename))
except Exception as err:
   # 웹캠이 없거나 인식할 수 없는 경우 오류 발생
   # 페이지에 액세스할 수 있는 권한을 부여해야됨.
  print(str(err))